# Week 3: Identify Risk Factors for Infection

<span style="color:red">
**UPDATE**

Thank you again for the previous analysis. We will next be publishing a public health advisory that warns of specific infection risk factors of which individuals should be aware. Please advise as to which population characteristics are associated with higher infection rates. 
</span>

Your goal for this notebook will be to identify key potential demographic and economic risk factors for infection by comparing the infected and uninfected populations.

## Imports

In [1]:
import cudf
import cuml
import cupy as cp

## Load Data

Begin by loading the data you've received about week 3 of the outbreak into a cuDF data frame. The data is located at `./data/week3.csv`. For this notebook you will need all columns of the data.

In [2]:
gdf = cudf.read_csv('./data/week3.csv')

gdf.head()

,age,sex,employment,infected
0,0,m,U,0.0
1,0,m,U,0.0
2,0,m,U,0.0
3,0,m,U,0.0
4,0,m,U,0.0


## Calculate Infection Rates by Employment Code

Convert the `infected` column to type `float32`. For people who are not infected, the float32 `infected` value should be `0.0`, and for infected people it should be `1.0`.

In [3]:
gdf["infected"] = gdf["infected"].astype('float32')

gdf.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 58479894 entries, 0 to 58479893
Data columns (total 4 columns):
 #   Column      Dtype
---  ------      -----
 0   age         int64
 1   sex         object
 2   employment  object
 3   infected    float32
dtypes: float32(1), int64(1), object(2)
memory usage: 1.2+ GB


Now, produce a list of employment types and their associated **rates** of infection, sorted from highest to lowest rate of infection.

**NOTE**: The infection **rate** for each employment type should be the percentage of total individuals within an employment type who are infected. Therefore, if employment type "X" has 1000 people, and 10 of them are infected, the infection **rate** would be .01. If employment type "Z" has 10,000 people, and 50 of them are infected, the infection rate would be .005, and would be **lower** than for type "X", even though more people within that employment type were infected.

In [4]:
# Create a dataframe
employment = cudf.DataFrame()

# Get ONLY infected
infected = gdf.loc[gdf["infected"] == 1].reset_index().drop(columns=["index"])
# Get all of the infected who are employed
employment["infected"] = infected.groupby("employment").count()["sex"]

# Get all of the employed
employment["total"] = gdf.groupby("employment").count()["sex"]

# Get employment type
employment = employment.reset_index()

# Get the % of infected employees
employment["rate"] = employment["infected"] / employment["total"]

employment

,employment,infected,total,rate
0,A,1178,305755,0.003853
1,"B, D, E",1837,486785,0.003774
2,C,10301,2653753,0.003882
3,F,6604,2075628,0.003182
4,G,17561,3549465,0.004948
5,H,4737,1398342,0.003388
6,I,16116,1556575,0.010354
7,J,4650,1180372,0.003939
8,K,5356,1122406,0.004772
9,L,1722,346470,0.004970


Finally, read in the employment codes guide from `./data/code_guide.csv` to interpret which employment types are seeing the highest rates of infection.infected_employed

In [5]:
codes = cudf.read_csv('./data/code_guide.csv')

codes

,Code,Field
0,A,"Agriculture, forestry & fishing"
1,"B, D, E","Mining, energy and water supply"
2,C,Manufacturing
3,F,Construction
4,G,"Wholesale, retail & repair of motor vehicles"
5,H,Transport & storage
6,I,Accommodation & food services
7,J,Information & communication
8,K,Financial & insurance activities
9,L,Real estate activities


## Calculate Infection Rates by Employment Code and Sex

We want to see if there is an effect of `sex` on infection rate, either in addition to `employment` or confounding it. Group by both `employment` and `sex` simultaneously to get the infection rate for the intersection of those categories.

In [6]:
infected_sex = gdf.loc[gdf["infected"] == 1].reset_index().drop(columns=["index"])

infected_sex = infected_sex.groupby(["employment", "sex"]).count().reset_index()

# infected_sex

infected_sex["rate"] = infected_sex["age"] / gdf.groupby(["employment", "sex"]).count().reset_index()["age"]
infected_sex

,employment,sex,age,infected,rate
0,A,f,616,616,0.007491
1,A,m,562,562,0.002514
2,"B, D, E",f,924,924,0.007973
3,"B, D, E",m,913,913,0.002462
4,C,f,5251,5251,0.007630
5,C,m,5050,5050,0.002569
6,F,f,2017,2017,0.007577
7,F,m,4587,4587,0.002535
8,G,f,12715,12715,0.007556
9,G,m,4846,4846,0.002596


## Quiz question - week 3
After completing Week 3, which occupation has the highest rate of the infection? **NOTE**: The infection rate for each employment type should be the percentage of total individuals within an employment type who are infected. Therefore, if employment type "X" has 1000 people, and 10 of them are infected, the infection rate would be .01. If employment type "Z" has 10,000 people, and 50 of them are infected, the infection rate would be .005, and would be lower than for type "X", even though more people within that employment type were infected.

In [7]:
# NOTE: 'employment' was already sorted 
employment["field"] = codes["Field"]
employment

# Get most infected employment type
employment.loc[employment["rate"] == employment["rate"].max()]

# Question answer: Human health & social work activities

,employment,infected,total,rate,field
14,Q,48505,3802602,0.012756,Human health & social work activities


## Take the Assessment

After completing the work above, visit the *Launch Section* web page that you used to launch this Jupyter Lab. Scroll down below where you launched Jupyter Lab, and answer the question *Week 3 Assessment*. You can view your overall progress in the assessment by visiting the same *Launch Section* page and clicking on the link to the *Progress* page. On the *Progress* page, if you have successfully answered all the assessment questions, you can click on *Generate Certificate* to receive your certificate in the course.

![launch_task_page](./images/launch_task_page.png)

<div align="center"><h2>Optional: Restart the Kernel</h2></div>

If you plan to continue work in other notebooks, please shutdown the kernel.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Next

Congratulations on all your work! Please continue to [the course conclusion](3-05_course_conclusion.ipynb).
